# Notebook to update list of indicators and functions in README.md
**Warning** developer use only!


In [1]:
import re

import inspect

import pandas as pd
import readme_renderer.rst

import project

from mintalib import core
from mintalib import functions
from mintalib import indicators

from mintalib.utils import sample_prices

from pprint import pformat


# TODO move list_functions, list_indicators to utils ?


In [2]:
def get_description(f):
    doc = f.__doc__ or ""
    return doc.strip().partition("\n")[0]


def get_info(func):
    name = func.__qualname__
    result = dict(name=name)
    params = list(inspect.signature(func).parameters.values())
    result = dict(name=name)
    if params and params[0].name in ('series', 'prices'):
        result['input'] = params[0].name
    description = get_description(func)
    result.update(description=description)
    return result


def list_functions():
    """ list functions """

    result = []
    for k, v in vars(functions).items():
        if not callable(v):
            continue

        if v.__name__.startswith("_"):
            continue

        params = list(inspect.signature(v).parameters.values())
        if params and params[0].name not in ('series', 'prices'):
            continue

        result.append(v)

    result = [get_info(f) for f in result]

    result = pd.DataFrame(result).set_index('name')

    return result


print(list_functions())

           input                            description
name                                                   
AVGPRICE  prices                          Average Price
TYPPRICE  prices                          Typical Price
WCLPRICE  prices                   Weighted Close Price
MIDPRICE  prices                              Mid Price
LOG       series                              Logarithm
EXP       series                            Exponential
ROC       series                         Rate of Change
DIFF      series                             Difference
MIN       series                        Rolling Minimum
MAX       series                        Rolling Maximum
SUM       series                            Rolling Sum
MAD       series                Mean Absolute Deviation
STDEV     series                     Standard Deviation
SMA       series                  Simple Moving Average
EMA       series             Exponential Moving Average
WMA       series                Weighted Moving 

In [3]:
def list_indicators():
    """ list indicators """

    def check_indicator(obj):
        if not isinstance(v, type):
            return False

        if v.__name__.startswith("_"):
            return False

        if not v.__name__.isupper():
            return False

        return True


    result = []

    for k, v in vars(indicators).items():
        if check_indicator(v):
            result.append(v)
            for k, v in vars(v).items():
                if check_indicator(v):
                    result.append(v)

    result = [get_info(f) for f in result]

    result = pd.DataFrame(result).set_index('name')

    return result


print(list_indicators())


                                          description
name                                                 
PRICE                                   Generic Price
AVGPRICE                      Average Price Indicator
TYPPRICE                      Typical Price Indicator
WCLPRICE               Weighted Close Price Indicator
MIDPRICE                          Mid Price Indicator
VOLUME                               Volume Indicator
LOG                                         Logarithm
EXP                                       Exponential
ROC                                    Rate of Change
DIFF                                       Difference
MIN                                   Rolling Minimum
MAX                                   Rolling Maximum
SUM                                       Rolling Sum
MAD                            Mean Absolue Deviation
STDEV                              Standard Deviation
SMA                             Simple Moving Average
EMA                        E

In [4]:
def backup_readme(verbose=True):
    readme = project.root.joinpath("README.md")
    backup = project.root.joinpath("README.bak")

    if verbose:
        print(f"Backing up {readme.name} ...")

    contents = readme.read_text()
    size = backup.write_text(contents)
    return size

backup_readme()


Backing up README.md ...


9867

In [5]:
def update_functions(verbose=True):
    data = list_functions().to_markdown()

    prefix = "<details>\n<summary>List of Functions</summary>\n"
    suffix = "</details>\n"

    pattern = prefix + "([^<>]+)" + suffix
    repl = prefix + "\n" + data + "\n\n" + suffix

    readme = project.root.joinpath("README.md")
    contents = readme.read_text()

    buffer, count = re.subn(pattern, repl, contents, flags=re.MULTILINE)

    if count != 1:
        raise RuntimeError("Cound not replace functions")

    if verbose:
        print(f"Updating {readme.name} ...")

    size = readme.write_text(buffer)

    return size


update_functions()



Updating README.md ...


9381

In [6]:
def update_indicators(verbose=True):
    data = list_indicators().to_markdown()

    prefix = "<details>\n<summary>List of Indicators</summary>\n"
    suffix = "</details>\n"

    pattern = prefix + "([^<>]+)" + suffix
    repl = prefix + "\n" + data + "\n\n" + suffix

    readme = project.root.joinpath("README.md")
    contents = readme.read_text()

    buffer, count = re.subn(pattern, repl, contents, flags=re.MULTILINE)

    if count != 1:
        raise RuntimeError("Cound not replace indicators")

    if verbose:
        print(f"Updating {readme.name} ...")

    size = readme.write_text(buffer)

    return size

update_indicators()

Updating README.md ...


9381